<a href="https://colab.research.google.com/github/nortonvanz/PySpark-Basics/blob/main/notebooks/Case_2_PySpark_com_DataFrame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Conferir se há 2 pastas no Arquivos, na lateral esquerda. Se não tiver, descomenta as 3 linhas abaixo:

#!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
#!tar xf spark-3.1.2-bin-hadoop2.7.tgz

!pip install -q findspark

# Se tiver erros mais abaixo criando o context (spark), ajustar para que a versão do pyspark seja a mesma do spark:
  # spark  -> v3.1.2
  # !pip freeze   -> pyspark==3.5.0

  #equiparar versões:
  #!pip install pyspark==3.1.2

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f


spark = SparkSession.builder \
    .master('local[*]') \
    .appName('Iniciando com Spark') \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

In [ ]:
from pyspark.sql.types import FloatType

# EXERCÍCIOS COM PYSPARK


1. Ler o arquivo chipotle.tsv?
2. Quantos produtos custam mais de $ 10,00?
3. Qual o preço de cada item?
4. Ordenar pelo nome do item
5. Qual foi a quantidade do item mais caro encomendado?
6. Quantas vezes um Veggie Salad Bowl foi pedido?
7. Quantas vezes alguém pediu mais de um refrigerante em lata?
8. Salvar em parquet

# 1 - Ler o arquivo chipotle.tsv?

In [ ]:
!wget --quiet --show-progress https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv

chipotle.tsv.2      100%[===================>] 356.42K  --.-KB/s    in 0.005s  


In [ ]:
df =spark.read.csv("chipotle.tsv", sep=r'\t', header=True, inferSchema=True )

In [ ]:
df.show()

+--------+--------+--------------------+--------------------+----------+
|order_id|quantity|           item_name|  choice_description|item_price|
+--------+--------+--------------------+--------------------+----------+
|       1|       1|Chips and Fresh T...|                NULL|    $2.39 |
|       1|       1|                Izze|        [Clementine]|    $3.39 |
|       1|       1|    Nantucket Nectar|             [Apple]|    $3.39 |
|       1|       1|Chips and Tomatil...|                NULL|    $2.39 |
|       2|       2|        Chicken Bowl|[Tomatillo-Red Ch...|   $16.98 |
|       3|       1|        Chicken Bowl|[Fresh Tomato Sal...|   $10.98 |
|       3|       1|       Side of Chips|                NULL|    $1.69 |
|       4|       1|       Steak Burrito|[Tomatillo Red Ch...|   $11.75 |
|       4|       1|    Steak Soft Tacos|[Tomatillo Green ...|    $9.25 |
|       5|       1|       Steak Burrito|[Fresh Tomato Sal...|    $9.25 |
|       5|       1| Chips and Guacamole|           

In [ ]:
df

DataFrame[order_id: int, quantity: int, item_name: string, choice_description: string, item_price: string]

In [ ]:
#ver os tipos de dados
df.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- item_name: string (nullable = true)
 |-- choice_description: string (nullable = true)
 |-- item_price: string (nullable = true)



In [ ]:
df.show(5)

+--------+--------+--------------------+--------------------+----------+
|order_id|quantity|           item_name|  choice_description|item_price|
+--------+--------+--------------------+--------------------+----------+
|       1|       1|Chips and Fresh T...|                NULL|    $2.39 |
|       1|       1|                Izze|        [Clementine]|    $3.39 |
|       1|       1|    Nantucket Nectar|             [Apple]|    $3.39 |
|       1|       1|Chips and Tomatil...|                NULL|    $2.39 |
|       2|       2|        Chicken Bowl|[Tomatillo-Red Ch...|   $16.98 |
+--------+--------+--------------------+--------------------+----------+
only showing top 5 rows



# 2 - Quantos produtos custam mais de $ 10,00?

In [ ]:
#Remover o sinal $ da coluna item_price, e conveter pra float:
df2 = df.select('*', f.translate('item_price', "$", "").cast(FloatType()).alias("new_item_price"))
df2.show(3)

+--------+--------+--------------------+------------------+----------+--------------+
|order_id|quantity|           item_name|choice_description|item_price|new_item_price|
+--------+--------+--------------------+------------------+----------+--------------+
|       1|       1|Chips and Fresh T...|              NULL|    $2.39 |          2.39|
|       1|       1|                Izze|      [Clementine]|    $3.39 |          3.39|
|       1|       1|    Nantucket Nectar|           [Apple]|    $3.39 |          3.39|
+--------+--------+--------------------+------------------+----------+--------------+
only showing top 3 rows



In [ ]:
#dropar coluna antiga:
df2 = df2.drop(f.col("item_price"))
df2.show(3)

+--------+--------+--------------------+------------------+--------------+
|order_id|quantity|           item_name|choice_description|new_item_price|
+--------+--------+--------------------+------------------+--------------+
|       1|       1|Chips and Fresh T...|              NULL|          2.39|
|       1|       1|                Izze|      [Clementine]|          3.39|
|       1|       1|    Nantucket Nectar|           [Apple]|          3.39|
+--------+--------+--------------------+------------------+--------------+
only showing top 3 rows



In [ ]:
#sobrescrevendo df:
df = df2
df.show(2)

+--------+--------+--------------------+------------------+--------------+
|order_id|quantity|           item_name|choice_description|new_item_price|
+--------+--------+--------------------+------------------+--------------+
|       1|       1|Chips and Fresh T...|              NULL|          2.39|
|       1|       1|                Izze|      [Clementine]|          3.39|
+--------+--------+--------------------+------------------+--------------+
only showing top 2 rows



In [ ]:
#poderia usar uma função também:
#UDF: User defined funcion, função que não existe no Spark, criada pelo user

#@f.udf
#def replace_money(money):
#  return float(money.replace("$", ""))

#df.select('*', replace_money("item_price").alias("new_item_price")).show(5)

In [ ]:
#filtrar produtos com mais de 10.0
df.filter(f.col("new_item_price") > 10.0).show(5)

+--------+--------+---------------+--------------------+--------------+
|order_id|quantity|      item_name|  choice_description|new_item_price|
+--------+--------+---------------+--------------------+--------------+
|       2|       2|   Chicken Bowl|[Tomatillo-Red Ch...|         16.98|
|       3|       1|   Chicken Bowl|[Fresh Tomato Sal...|         10.98|
|       4|       1|  Steak Burrito|[Tomatillo Red Ch...|         11.75|
|       7|       1|   Chicken Bowl|[Fresh Tomato Sal...|         11.25|
|      12|       1|Chicken Burrito|[[Tomatillo-Green...|         10.98|
+--------+--------+---------------+--------------------+--------------+
only showing top 5 rows



In [ ]:
#acima, trabalhamos com o chamado "SQL FUNCTION", usando a lib: pyspark.sql.functions (como usamos o Pandas..)

#outra forma de trabalhar é com o chamado "SPARK SQL", usando SQL nativo diretamente, confrome abaixo:

In [ ]:
#criar uma view "chipotle" a partir de df:
df.createOrReplaceTempView("chipotle")

In [ ]:
#agora, podemos usar SQL para consultar:
spark.sql("select * from chipotle where new_item_price > 10.0").show(10)

+--------+--------+---------------+--------------------+--------------+
|order_id|quantity|      item_name|  choice_description|new_item_price|
+--------+--------+---------------+--------------------+--------------+
|       2|       2|   Chicken Bowl|[Tomatillo-Red Ch...|         16.98|
|       3|       1|   Chicken Bowl|[Fresh Tomato Sal...|         10.98|
|       4|       1|  Steak Burrito|[Tomatillo Red Ch...|         11.75|
|       7|       1|   Chicken Bowl|[Fresh Tomato Sal...|         11.25|
|      12|       1|Chicken Burrito|[[Tomatillo-Green...|         10.98|
|      19|       1|  Barbacoa Bowl|[Roasted Chili Co...|         11.75|
|      20|       1|   Chicken Bowl|[Roasted Chili Co...|         11.25|
|      20|       1|  Steak Burrito|[Fresh Tomato Sal...|         11.75|
|      21|       1|Chicken Burrito|[Tomatillo-Red Ch...|         10.98|
|      24|       1|Chicken Burrito|[Roasted Chili Co...|         10.98|
+--------+--------+---------------+--------------------+--------

# 3 - Qual o preço de cada item?

In [ ]:
#número de itens
df.count()

4622

In [ ]:
#preço de cada item sem duplicidade
df.dropDuplicates(["item_name", "choice_description", "new_item_price"]).count()

1960

In [ ]:
#preços de cada item + descrição sem duplicidade
df_prices = df.dropDuplicates(["item_name", "choice_description", "new_item_price"])
df_prices.show(5)

+--------+--------+------------------+--------------------+--------------+
|order_id|quantity|         item_name|  choice_description|new_item_price|
+--------+--------+------------------+--------------------+--------------+
|     518|       1|   Chicken Burrito|[[Roasted Chili C...|         10.98|
|    1313|       1|  Barbacoa Burrito|[Fresh Tomato Sal...|         11.48|
|    1462|       2|Chicken Soft Tacos|[[Tomatillo-Green...|         21.96|
|    1666|       1|      Chicken Bowl|[Tomatillo Red Ch...|          8.75|
|    1775|       1|       Veggie Bowl|[Fresh Tomato Sal...|         11.25|
+--------+--------+------------------+--------------------+--------------+
only showing top 5 rows



# 4 Ordenar pelo nome do item

In [ ]:
#SPARK SQL
spark.sql('''

          SELECT DISTINCT ITEM_NAME
          FROM CHIPOTLE
          ORDER BY ITEM_NAME

          ''' ).show(5)

+--------------------+
|           ITEM_NAME|
+--------------------+
|   6 Pack Soft Drink|
|       Barbacoa Bowl|
|    Barbacoa Burrito|
|Barbacoa Crispy T...|
| Barbacoa Salad Bowl|
+--------------------+
only showing top 5 rows



In [ ]:
# SQL FUNCTION

#df.sort('ITEM_NAME').show(5)  ->  tem que ser distinto:
df.orderBy(f.col('ITEM_NAME').asc()).show(5)

+--------+--------+-----------------+------------------+----------+
|order_id|quantity|        item_name|choice_description|item_price|
+--------+--------+-----------------+------------------+----------+
|     154|       1|6 Pack Soft Drink|            [Coke]|    $6.49 |
|     298|       1|6 Pack Soft Drink|          [Nestea]|    $6.49 |
|     168|       1|6 Pack Soft Drink|       [Diet Coke]|    $6.49 |
|     129|       1|6 Pack Soft Drink|          [Sprite]|    $6.49 |
|     182|       1|6 Pack Soft Drink|       [Diet Coke]|    $6.49 |
+--------+--------+-----------------+------------------+----------+
only showing top 5 rows



In [ ]:
#selecionando 1 coluna:
df.select("item_name").show(3)

+--------------------+
|           item_name|
+--------------------+
|Chips and Fresh T...|
|                Izze|
|    Nantucket Nectar|
+--------------------+
only showing top 3 rows



In [ ]:
#dropando itens duplicados, selecionando uma coluna e ordenando asc:
df_aux = df.dropDuplicates(["item_name"])

df_aux.select("item_name").sort(f.col("item_name").asc()).show(5)

+--------------------+
|           item_name|
+--------------------+
|   6 Pack Soft Drink|
|       Barbacoa Bowl|
|    Barbacoa Burrito|
|Barbacoa Crispy T...|
| Barbacoa Salad Bowl|
+--------------------+
only showing top 5 rows



# 5 Qual foi a quantidade do item mais caro encomendado?

In [ ]:
df.show(5)

+--------+--------+--------------------+--------------------+----------+
|order_id|quantity|           item_name|  choice_description|item_price|
+--------+--------+--------------------+--------------------+----------+
|       1|       1|Chips and Fresh T...|                NULL|    $2.39 |
|       1|       1|                Izze|        [Clementine]|    $3.39 |
|       1|       1|    Nantucket Nectar|             [Apple]|    $3.39 |
|       1|       1|Chips and Tomatil...|                NULL|    $2.39 |
|       2|       2|        Chicken Bowl|[Tomatillo-Red Ch...|   $16.98 |
+--------+--------+--------------------+--------------------+----------+
only showing top 5 rows



In [ ]:
#SPARK SQK
spark.sql('''
          SELECT
            QUANTITY,
            MAX(new_item_price) AS MAX_PRICE
          FROM CHIPOTLE
          GROUP BY QUANTITY
          ORDER BY MAX_PRICE DESC
          LIMIT 1
        ''').show(5)

+--------+---------+
|QUANTITY|MAX_PRICE|
+--------+---------+
|      15|    44.25|
+--------+---------+



In [ ]:
# item mais caro
df.select(f.max(df.new_item_price)).show()

+-------------------+
|max(new_item_price)|
+-------------------+
|              44.25|
+-------------------+



In [ ]:
#SQL FUNCTION

In [ ]:
df.show(5)

+--------+--------+--------------------+--------------------+----------+
|order_id|quantity|           item_name|  choice_description|item_price|
+--------+--------+--------------------+--------------------+----------+
|       1|       1|Chips and Fresh T...|                NULL|    $2.39 |
|       1|       1|                Izze|        [Clementine]|    $3.39 |
|       1|       1|    Nantucket Nectar|             [Apple]|    $3.39 |
|       1|       1|Chips and Tomatil...|                NULL|    $2.39 |
|       2|       2|        Chicken Bowl|[Tomatillo-Red Ch...|   $16.98 |
+--------+--------+--------------------+--------------------+----------+
only showing top 5 rows



In [ ]:
new_df = df.groupBy("quantity").agg(f.max("new_item_price"))
new_df.sort(f.col("max(new_item_price)").desc()).show(1)

+--------+-------------------+
|quantity|max(new_item_price)|
+--------+-------------------+
|      15|              44.25|
+--------+-------------------+
only showing top 1 row



# 6 - Quantas vezes um Veggie Salad Bowl foi pedido?

In [ ]:
df.show(5)

+--------+--------+--------------------+--------------------+--------------+
|order_id|quantity|           item_name|  choice_description|new_item_price|
+--------+--------+--------------------+--------------------+--------------+
|       1|       1|Chips and Fresh T...|                NULL|          2.39|
|       1|       1|                Izze|        [Clementine]|          3.39|
|       1|       1|    Nantucket Nectar|             [Apple]|          3.39|
|       1|       1|Chips and Tomatil...|                NULL|          2.39|
|       2|       2|        Chicken Bowl|[Tomatillo-Red Ch...|         16.98|
+--------+--------+--------------------+--------------------+--------------+
only showing top 5 rows



In [ ]:
#SPARK SQL
spark.sql('''
          SELECT
            ITEM_NAME,
            COUNT(ORDER_ID) AS VENDAS
          FROM CHIPOTLE
          WHERE ITEM_NAME = 'Veggie Salad Bowl'
          GROUP BY ITEM_NAME
        ''').show(5)

+-----------------+------+
|        ITEM_NAME|VENDAS|
+-----------------+------+
|Veggie Salad Bowl|    18|
+-----------------+------+



In [ ]:
#SQL FUNCTION

In [ ]:
df.filter(f.col("item_name") == "Veggie Salad Bowl").count()

18

In [ ]:
new_df_2 = df.filter(f.col("item_name") == "Veggie Salad Bowl")

In [ ]:
new_df_2.groupBy("item_name").agg(f.sum("quantity")).show()

+-----------------+-------------+
|        item_name|sum(quantity)|
+-----------------+-------------+
|Veggie Salad Bowl|           18|
+-----------------+-------------+



# 7 - Quantas vezes alguém pediu mais de um refrigerante em lata : Canned Soda?

In [ ]:
df.show(5)

+--------+--------+--------------------+--------------------+--------------+
|order_id|quantity|           item_name|  choice_description|new_item_price|
+--------+--------+--------------------+--------------------+--------------+
|       1|       1|Chips and Fresh T...|                NULL|          2.39|
|       1|       1|                Izze|        [Clementine]|          3.39|
|       1|       1|    Nantucket Nectar|             [Apple]|          3.39|
|       1|       1|Chips and Tomatil...|                NULL|          2.39|
|       2|       2|        Chicken Bowl|[Tomatillo-Red Ch...|         16.98|
+--------+--------+--------------------+--------------------+--------------+
only showing top 5 rows



In [ ]:
#SQL FUNCTION
df.filter((f.col("item_name") == "Canned Soda") & (f.col("quantity") > 1)).count()

20

In [ ]:
#SPARK SQL
spark.sql('''
          SELECT
            ITEM_NAME,
            COUNT(ORDER_ID) AS VENDAS
          FROM CHIPOTLE
          WHERE ITEM_NAME = 'Canned Soda'
          AND QUANTITY > 1
          GROUP BY ITEM_NAME
        ''').show(5)

+-----------+------+
|  ITEM_NAME|VENDAS|
+-----------+------+
|Canned Soda|    20|
+-----------+------+



# 8 - Salvar em parquet

In [ ]:
#salva compactado

df.write.mode("overwrite").parquet("./export/df_chipotle.parquet")

#pode criar vários arquivos, com inicial "part", a depender do tamanho

# 9 - Ler do parquet

In [ ]:
df_x = spark.read.parquet("./export/df_chipotle.parquet")

In [ ]:
df_x.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- item_name: string (nullable = true)
 |-- choice_description: string (nullable = true)
 |-- new_item_price: float (nullable = true)



In [ ]:
df_x.show(5)

+--------+--------+--------------------+--------------------+--------------+
|order_id|quantity|           item_name|  choice_description|new_item_price|
+--------+--------+--------------------+--------------------+--------------+
|       1|       1|Chips and Fresh T...|                NULL|          2.39|
|       1|       1|                Izze|        [Clementine]|          3.39|
|       1|       1|    Nantucket Nectar|             [Apple]|          3.39|
|       1|       1|Chips and Tomatil...|                NULL|          2.39|
|       2|       2|        Chicken Bowl|[Tomatillo-Red Ch...|         16.98|
+--------+--------+--------------------+--------------------+--------------+
only showing top 5 rows

